# Wrapping the Fine-Tuned Model into an Agentic pipeline (LangChain)

## Dependencies

### Modules 

In [ ]:
%pip install langchain

In [2]:
%pip install langchain_community

In [3]:
%pip install langchain-huggingface

In [ ]:
%pip install -U langchain-huggingface transformers

### Imports

In [5]:
import langchain, langchain_core, langchain_community
print(langchain.__version__, langchain_core.__version__, langchain_community.__version__)


0.3.26 0.3.74 0.3.27


In [47]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import tool
from langchain.agents import create_react_agent, AgentExecutor, create_tool_calling_agent
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_huggingface import HuggingFacePipeline 
from langchain_huggingface import ChatHuggingFace
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM, AutoModelForCausalLM
import numpy as np
import torch

In [7]:
use_gpu = torch.cuda.is_available()
device = 0 if use_gpu else -1

## Agent

In [34]:
gen_model_path = "Qwen/Qwen2.5-1.5B-Instruct"

In [ ]:
gen_tok = AutoTokenizer.from_pretrained(gen_model_path)
gen_model = AutoModelForCausalLM.from_pretrained(gen_model_path)
gen_pipe = pipeline("text-generation",
                    model=gen_model, 
                    tokenizer=gen_tok, 
                    max_new_tokens=256,
                    
                    device=device
                   )

In [40]:
llm = ChatHuggingFace(llm=HuggingFacePipeline(pipeline=gen_pipe))

In [12]:
gen_pipe("hello!")

[{'generated_text': 'hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello! hello!'}]

In [13]:
classifier_path = "/kaggle/input/classifier-model/bert-spam-ham-classifier-full_dataset/checkpoint-13086"
cls_model = AutoModelForSequenceClassification.from_pretrained(classifier_path)
cls_tokenizer = AutoTokenizer.from_pretrained(classifier_path)

In [14]:
clf = pipeline("text-classification", 
               model=cls_model, 
               tokenizer=cls_tokenizer, 
               device=device
              )

Device set to use cuda:0


In [41]:
clf("hello click here")

[{'label': 'LABEL_1', 'score': 0.6220700144767761}]

In [16]:
@tool("classify_spam_ham")
def classify_spam_ham(text: str) -> str:
    """
    Classify the given message as spam or ham.
    """
    pred = clf(text)[0]["label"]
    return "spam" if pred == "LABEL_1" else "ham"

In [20]:
tools = [classify_spam_ham]

In [84]:
system = """
You are a terminal assistant.

RULES:
- Only call the tool `classify_spam_ham` if the user EXPLICITLY asks to classify,
  OR they use trigger phrases like: "classify", "is this spam", "spam or ham",
  "label this", "check for spam", or the input starts with "Classify this:" or "Classify:".
- If the user sends a greeting or small talk (e.g., "hello", "hi"), DO NOT call any tool.
- When you do call the tool, after it returns, output EXACTLY the tool's result
  in lowercase (`spam` or `ham`), with no extra words or punctuation.
"""

In [53]:
#prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a terminal agent. You can call tools.\n\n"
     "Available tools:\n{tools}\n\nUse exactly: {tool_names}\n"
     "If the user pastes a message, prefer classification."),
    ("ai", "{agent_scratchpad}"),
    ("human", "{input}")
])

In [85]:
prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    MessagesPlaceholder("chat_history"),      
    ("human", "{input}"),                     
    MessagesPlaceholder("agent_scratchpad"),
])

In [ ]:
#agent = create_react_agent(llm, tools, prompt=prompt)

In [86]:
agent = create_tool_calling_agent(llm, tools, prompt)

In [87]:
agent_exec = AgentExecutor(agent=agent, 
                           tools=tools, 
                           verbose=True, 
                           return_intermediate_steps=True)

In [88]:
res = agent_exec.invoke({"input": "Classify this: You won $10,000! Claim now.", "chat_history": []})
print(res["output"])



> Entering new AgentExecutor chain...
spam

> Finished chain.
spam


In [93]:
res = agent_exec.invoke({"input": "Who is Michael Jordan?", "chat_history": []})
print(res["output"])



> Entering new AgentExecutor chain...
Michael Jordan was an American professional basketball player who had an exceptional career. He played for the Chicago Bulls and led them to six NBA championships from 1991 to 1993. Jordan also won three Olympic gold medals as part of the U.S. men's national basketball team. His skills on both ends of the court made him one of the greatest players in the history of basketball.

> Finished chain.
Michael Jordan was an American professional basketball player who had an exceptional career. He played for the Chicago Bulls and led them to six NBA championships from 1991 to 1993. Jordan also won three Olympic gold medals as part of the U.S. men's national basketball team. His skills on both ends of the court made him one of the greatest players in the history of basketball.
